In [10]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh -y

# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager

if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/balazik/ComfyUI-PuLID-Flux.git
%cd ComfyUI-PuLID-Flux
!pip install insightface
!pip install -r requirements.txt

/kaggle/working
fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/kaggle/working/ComfyUI
mkdir: cannot create directory '/kaggle/temp': File exists
mkdir: cannot create directory '/kaggle/temp/temp-models': File exists

Looking for: ['openssh']

nvidia/linux-64                                             Using cache
nvidia/noarch                                               Using cache
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
[+] 0.1s
rapidsai/linux-64   ━━━━━━━━━━━━╸━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
rapidsai/noarch     ━━━━━━━━━━━━╸━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
nodefaults/linux-64 ━━━━━━━━╸━━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
nodefaults/noarch   ━━━╸━━━━━━━━━━━━━━━╸━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1snodefaults/linux-64                                           No change
rapidsai/linux-64     

# CheckPoint

In [ ]:
# Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/198530'
model_name = 'JuggernautXL.safetensors'

%cd $checkpoints
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Lora

In [ ]:
# Install a LoRA in permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

%cd /kaggle/working/ComfyUI/models/loras
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Launch With RemoteMoe

In [ ]:
# Starting with RemoteMoe

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!python /kaggle/working/ComfyUI/main.py & ssh -R 80:127.0.0.1:8188 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe 

# Launch with Pinggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

waiting for output
Allocated port 9 for remote forward to localhost:8188
waiting for output
Total VRAM 15095 MB, total RAM 32110 MB
pytorch version: 2.4.0
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
😁 😁 😁
URL: http://rndyw-34-170-255-35.a.free.pinggy.link
😁 😁 😁
7=)0]8;;\                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         ┌────────────────────────────┐

In [22]:
!rm -rf /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager
!rm -rf ~/.cache
!pip cache purge
!conda clean --all -y


In [3]:
import os
import requests

# Define the URL and the target directory
url = "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf"
target_dir = "/kaggle/working/ComfyUI/models/unet"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Define the target file path
target_file_path = os.path.join(target_dir, "flux1-dev-Q8_0.gguf")

# Download the file
response = requests.get(url)
with open(target_file_path, "wb") as f:
    f.write(response.content)

print(f"File downloaded and saved to: {target_file_path}")


File downloaded and saved to: /kaggle/working/ComfyUI/models/unet/flux1-dev-Q8_0.gguf


In [4]:
import os
import requests

# Define the URL and the target directory
url = "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true"
target_dir = "/kaggle/working/ComfyUI/models/clip"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Define the target file path
target_file_path = os.path.join(target_dir, "clip_l.safetensors")

# Download the file
response = requests.get(url)
with open(target_file_path, "wb") as f:
    f.write(response.content)

print(f"File downloaded and saved to: {target_file_path}")


File downloaded and saved to: /kaggle/working/ComfyUI/models/clip/clip_l.safetensors


In [5]:
import os
import requests

# Define the URL and the target directory
url = "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q8_0.gguf?download=true"
target_dir = "/kaggle/working/ComfyUI/models/clip"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Define the target file path
target_file_path = os.path.join(target_dir, "t5-v1_1-xxl-encoder-Q8_0.gguf")

# Download the file
response = requests.get(url)
with open(target_file_path, "wb") as f:
    f.write(response.content)

print(f"File downloaded and saved to: {target_file_path}")


File downloaded and saved to: /kaggle/working/ComfyUI/models/clip/t5-v1_1-xxl-encoder-Q8_0.gguf


In [6]:
import os
import requests

# Define the URL and the target directory
url = "https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/vae/diffusion_pytorch_model.safetensors?download=true"
target_dir = "/kaggle/working/ComfyUI/models/vae"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Define the target file path
target_file_path = os.path.join(target_dir, "diffusion_pytorch_model.safetensors")

# Download the file
response = requests.get(url)
with open(target_file_path, "wb") as f:
    f.write(response.content)

print(f"File downloaded and saved to: {target_file_path}")


File downloaded and saved to: /kaggle/working/ComfyUI/models/vae/diffusion_pytorch_model.safetensors


In [8]:
import os
import requests

# Define the URL and the target directory
url = "https://huggingface.co/guozinan/PuLID/resolve/main/pulid_flux_v0.9.0.safetensors?download=true"
target_dir = "/kaggle/working/ComfyUI/models/pulid"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Define the target file path
target_file_path = os.path.join(target_dir, "pulid_flux_v0.9.0.safetensors")

# Download the file
response = requests.get(url)
with open(target_file_path, "wb") as f:
    f.write(response.content)

print(f"File downloaded and saved to: {target_file_path}")


File downloaded and saved to: /kaggle/working/ComfyUI/models/pulid/pulid_flux_v0.9.0.safetensors
